## [pyKNEEr](https://github.com/sbonaretti/pyKNEEr)

# [Relaxometry of Femoral Knee Cartilage](https://sbonaretti.github.io/pyKNEEr/relaxometry.html)
# Exponential and linear fitting
Content under Creative Commons Attribution license CC-BY-NC-SA 4.0   
Code under GNU-GPL v3 License  
© 2019 Serena Bonaretti
---

- *Exponential* fitting is computationally expensive but more accurate
- *Linear fitting* is faster as data are transformed to their log and then linearly interpolated. However, linear fitting is less accurate because the nonlinear logaritmic transform provides larger weight to outliers  

The fitting is computed:  
- *directly* on the acquired images or after *rigid registration* of the following echo to the first echo  
- voxel-wise, i.e. for each voxel the Echo Times (dicom tag: (0018,0081)) are the x-variable and the voxel intensities in each acquisition are the y-variable
- only in the mask volume to have short computation time

## Import packages

In [ ]:
from pykneer import pykneer_io         as io
from pykneer import relaxometry_for_nb as rel
import itk
import itkwidgets
from itkwidgets import view

## Image information

Inputs:   
- `input_file_name` contains the list of the images used to calculate the relaxation maps
- `method` is `0` if fitting is linear, `1` if fitting is exponential
- `registration_flag` is `0` for no registration, `1` for rigid registration
- `output_file_name` contains average and standard deviation of the fitting maps 

In [ ]:
input_file_name   = "./image_list_relaxometry_fitting.txt"
method_flag       = 1 # 0 = linear, 1 = exponential
registration_flag = 1 # 0 = no rigid registration, 1 = execute rigid registration
n_of_cores        = 1
output_file_name  = "exp_fit_aligned.csv"

#### Read image data
- `image_data` is a dictionary (or struct), where each cell corresponds to an image. For each image, information such as paths and file names are stored  

In [ ]:
image_data = io.load_image_data_fitting(input_file_name, method_flag, registration_flag)

## Calculate fitting maps

#### Align acquisitions 
Images are aligned rigidly to remove occational subject motion among acquisitions   

Note: This step is optional and can be skipped, given that:
 - When images are aligned, the fitting is calculated on interpolated values obtained with rigid registration
 - When images are not aligned, the fitting is calculated on original intensities, but images might not be aligned

In [ ]:
if registration_flag == 1:
    rel.align_acquisitions(image_data, n_of_cores)

#### Compute the fitting

In [ ]:
rel.calculate_fitting_maps(image_data, n_of_cores)

## Visualize fitting maps

#### 2D MAP: For each image, fitting maps at medial and lateral compartments and flattened map
Set the variable `view_modality` to `0` to show one slice of the image (static and faster rendering) or to `1` to scroll though the image (interactive and slower rendering). Note that when using interactive rendering, images might NOT be saved  for when reopening the notebook.

In [ ]:
view_modality = 1; # 0 for static, 1 for interactive
fig = rel.show_fitting_maps(image_data, view_modality)
display(fig)

#### 3D MAP: Interactive rendering of fitting maps
(The error message "Error creating widget: could not find model" can appear when the notebook is moved to a different folder)  
(If you get "Error displaying widget", [install](https://github.com/InsightSoftwareConsortium/itk-jupyter-widgets) the widget in the terminal by typing: `jupyter labextension install @jupyter-widgets/jupyterlab-manager itk-jupyter-widgets` and restart JupyterLab)

In [ ]:
# ID of the map to visualize (The ID is the one in the 2D visualization above)
image_ID = 1 -1 # -1 because counting starts from 0

# read image
file_name = image_data[image_ID]["relaxometry_folder"] + image_data[image_ID]["map_file_name"]
image = itk.imread(file_name)

# view
viewer = view(image, gradient_opacity=0.0, ui_collapsed=False, shadow=False)
viewer

#### GRAPH: Dots represent the average value of fitting maps per image; bars represents the standard deviation

In [ ]:
rel.show_fitting_graph(image_data)

#### TABLE: Average and standard deviation of fitting maps per image
The table is saved as a .csv file for subsequent analyisis

In [ ]:
rel.show_fitting_table(image_data, output_file_name)

### References

[1] Borthakur A., Wheaton A.J., Gougoutas A.J., Akella S.V., Regatte R.R., Charagundla S.R., Reddy R. [*In vivo measurement of T1rho dispersion in the human brain at 1.5 tesla.*](https://www.ncbi.nlm.nih.gov/pubmed/15065163) J Magn Reson Imaging. Apr;19(4):403-9. 2004.  
[2] Li X., Benjamin Ma C., Link T.M., Castillo D.D., Blumenkrantz G., Lozano J., Carballido-Gamio J., Ries M., Majumdar S. [*In vivo T1ρ and T2 mapping of articular cartilage in osteoarthritis of the knee using 3 T MRI.*](https://www.ncbi.nlm.nih.gov/pubmed/17307365) Osteoarthritis Cartilage. Jul;15(7):789-97. 2007.  

### Dependencies

In [ ]:
%load_ext watermark
print ("elastix 4.8.0\n")
%watermark -v -m -p pykneer,SimpleITK,matplotlib,numpy,pandas,scipy,itkwidgets,multiprocessing,ipywidgets,watermark
print (" ")
%watermark -u -n -t -z 